In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression 
import statsmodels.formula.api as sm

In [3]:
df = pd.read_csv("Demand.csv",index_col=0)


FileNotFoundError: File b'Demand.csv' does not exist

In [113]:
df.head()

City Mapped_Sales_Type  Mapped_Item_Code Transaction_Date  \
0  Hyderabad         Take Away              9978       2018-07-31   
1  Hyderabad         Take Away              4064       2018-07-31   
2  Hyderabad         Main Hall               965       2018-07-31   
3  Hyderabad           Express              9978       2018-07-31   
4  Hyderabad           Express              1045       2018-07-31   

   Quantity_Sold  Median_Price  Average_Tax_Percent  Effective_Price  \
0            167         187.0                 0.05       163.700000   
1            368         602.0                 0.05       584.329105   
2            694         263.0                 0.05       244.835163   
3            140         263.0                 0.05       243.500000   
4            139         392.0                 0.05       378.960000   

   Percent_Quantity_With_Discount  Avg_Discount_Percent_On_Discounted_Items  \
0                             NaN                                       NaN   
1                        0.027237                                      1.05   
2                             NaN                                       NaN   
3                             NaN                                       NaN   
4                             NaN                                       NaN   

         ...          Day  Car_sales F_and_B_Inflation  \
0        ...          Tue    7559.28             143.7   
1        ...          Tue    7559.28             143.7   
2        ...          Tue    7559.28             143.7   
3        ...          Tue    7559.28             143.7   
4        ...          Tue    7559.28             143.7   

   F_and_B_Inflation_Adjusted_Effective_Price  \
0                                    0.832985   
1                                    3.760119   
2                                    1.397600   
3                                    1.388309   
4                                    2.330967   

   F_and_B_Inflation_Adjusted_Median_Price  First_Week_Of_The_Month_Dummy  \
0                                 0.793319                              0   
1                                 3.681280                              0   
2                                 1.322199                              0   
3                                 1.322199                              0   
4                                 2.219903                              0   

   Last_Week_Of_The_Month_Dummy  Day_Of_The_Week_Seasonality  \
0                             1                         0.86   
1                             1                         0.83   
2                             1                         1.08   
3                             1                         0.78   
4                             1                         0.73   

   Month_Of_The_Year_Seasonality  Two_Wheeler_sales  
0                           0.91           37307.52  
1                           0.91           37307.52  
2                           1.08           37307.52  
3                           0.08           37307.52  
4                           0.08           37307.52  

[5 rows x 22 columns]

In [114]:
df.dtypes

City                                           object
Mapped_Sales_Type                              object
Mapped_Item_Code                                int64
Transaction_Date                               object
Quantity_Sold                                   int64
Median_Price                                  float64
Average_Tax_Percent                           float64
Effective_Price                               float64
Percent_Quantity_With_Discount                float64
Avg_Discount_Percent_On_Discounted_Items      float64
Year                                            int64
Month                                           int64
Day                                            object
Car_sales                                     float64
F_and_B_Inflation                             float64
F_and_B_Inflation_Adjusted_Effective_Price    float64
F_and_B_Inflation_Adjusted_Median_Price       float64
First_Week_Of_The_Month_Dummy                   int64
Last_Week_Of_The_Month_Dummy

In [115]:
df['Mapped_Item_Code'] = df['Mapped_Item_Code'].astype(str)

In [116]:
df['Transaction_Date'] = pd.to_datetime(df['Transaction_Date'])

In [150]:
df['Transaction_Date'].min()

Timestamp('2016-08-01 00:00:00')

In [1]:
df = df.loc[(df['Transaction_Date'] >= '2016-08-01'), :]

NameError: name 'df' is not defined

In [118]:
df.columns

Index(['City', 'Mapped_Sales_Type', 'Mapped_Item_Code', 'Transaction_Date',
       'Quantity_Sold', 'Median_Price', 'Average_Tax_Percent',
       'Effective_Price', 'Percent_Quantity_With_Discount',
       'Avg_Discount_Percent_On_Discounted_Items', 'Year', 'Month', 'Day',
       'Car_sales', 'F_and_B_Inflation',
       'F_and_B_Inflation_Adjusted_Effective_Price',
       'F_and_B_Inflation_Adjusted_Median_Price',
       'First_Week_Of_The_Month_Dummy', 'Last_Week_Of_The_Month_Dummy',
       'Day_Of_The_Week_Seasonality', 'Month_Of_The_Year_Seasonality',
       'Two_Wheeler_sales'],
      dtype='object')

In [119]:
df['Mapped_Sales_Type'].unique()

array(['Take Away', 'Main Hall', 'Express', 'Delivery', 'Dine-in', 'Bakery'], dtype=object)

In [120]:
# df['Avg_Discount_Percent_On_Discounted_Items'] = np.where(df['P*Q'].notnull(), (df['Discount_Amount']/df['P*Q'])*df['Tax%'] , np.nan )

In [121]:
df = df.loc[df['Avg_Discount_Percent_On_Discounted_Items'] < 1.0, :]

In [122]:
df.loc[df['Avg_Discount_Percent_On_Discounted_Items'] >= 1.0]

Empty DataFrame
Columns: [City, Mapped_Sales_Type, Mapped_Item_Code, Transaction_Date, Quantity_Sold, Median_Price, Average_Tax_Percent, Effective_Price, Percent_Quantity_With_Discount, Avg_Discount_Percent_On_Discounted_Items, Year, Month, Day, Car_sales, F_and_B_Inflation, F_and_B_Inflation_Adjusted_Effective_Price, F_and_B_Inflation_Adjusted_Median_Price, First_Week_Of_The_Month_Dummy, Last_Week_Of_The_Month_Dummy, Day_Of_The_Week_Seasonality, Month_Of_The_Year_Seasonality, Two_Wheeler_sales]
Index: []

[0 rows x 22 columns]

In [123]:
dict_aggr1 = {'Quantity_Sold':'sum', 'Median_Price':'median', 'Effective_Price':'median'}

In [124]:
df_agg = df.groupby(['City', 'Mapped_Sales_Type', 'Mapped_Item_Code', 'Transaction_Date']).agg(dict_aggr1).reset_index()

In [125]:
df_agg[:3]

City Mapped_Sales_Type Mapped_Item_Code Transaction_Date  \
0  Hyderabad            Bakery             4059       2018-06-14   
1  Hyderabad            Bakery             4060       2018-06-14   
2  Hyderabad            Bakery             4063       2018-06-14   

   Quantity_Sold  Median_Price  Effective_Price  
0            109         359.0           314.27  
1            110         378.0           332.22  
2            110         732.0           666.76

In [126]:
df.loc[df['Median_Price'].isnull()]

Empty DataFrame
Columns: [City, Mapped_Sales_Type, Mapped_Item_Code, Transaction_Date, Quantity_Sold, Median_Price, Average_Tax_Percent, Effective_Price, Percent_Quantity_With_Discount, Avg_Discount_Percent_On_Discounted_Items, Year, Month, Day, Car_sales, F_and_B_Inflation, F_and_B_Inflation_Adjusted_Effective_Price, F_and_B_Inflation_Adjusted_Median_Price, First_Week_Of_The_Month_Dummy, Last_Week_Of_The_Month_Dummy, Day_Of_The_Week_Seasonality, Month_Of_The_Year_Seasonality, Two_Wheeler_sales]
Index: []

[0 rows x 22 columns]

In [127]:
df.loc[df['Effective_Price'].notnull()].head()

City Mapped_Sales_Type Mapped_Item_Code Transaction_Date  \
5   Hyderabad          Delivery             4056       2018-07-31   
10  Hyderabad          Delivery             4061       2018-07-31   
20  Hyderabad           Dine-in              966       2018-07-31   
24  Hyderabad           Dine-in              965       2018-07-31   
26  Hyderabad          Delivery             4068       2018-07-31   

    Quantity_Sold  Median_Price  Average_Tax_Percent  Effective_Price  \
5             617         322.0                 0.05       305.354585   
10            144         942.0                 0.05       942.631515   
20           1250         392.0                 0.05       378.840053   
24           1934         362.0                 0.05       347.258327   
26            148         622.0                 0.05       618.898649   

    Percent_Quantity_With_Discount  Avg_Discount_Percent_On_Discounted_Items  \
5                         0.003953                                  0.105020   
10                        0.151515                                  0.105006   
20                        0.003512                                  0.105008   
24                        0.001646                                  0.420000   
26                        0.027027                                  0.105009   

          ...          Day  Car_sales F_and_B_Inflation  \
5         ...          Tue    7559.28             143.7   
10        ...          Tue    7559.28             143.7   
20        ...          Tue    7559.28             143.7   
24        ...          Tue    7559.28             143.7   
26        ...          Tue    7559.28             143.7   

    F_and_B_Inflation_Adjusted_Effective_Price  \
5                                     1.818751   
10                                    6.253525   
20                                    2.330133   
24                                    2.110357   
26                                    4.000686   

    F_and_B_Inflation_Adjusted_Median_Price  First_Week_Of_The_Month_Dummy  \
5                                  1.732777                              0   
10                                 6.047321                              0   
20                                 2.219903                              0   
24                                 2.011134                              0   
26                                 3.820459                              0   

    Last_Week_Of_The_Month_Dummy  Day_Of_The_Week_Seasonality  \
5                              1                         0.82   
10                             1                         0.75   
20                             1                         0.92   
24                             1                         0.89   
26                             1                         0.64   

    Month_Of_The_Year_Seasonality  Two_Wheeler_sales  
5                            1.06           37307.52  
10                           1.06           37307.52  
20                           1.09           37307.52  
24                           1.09           37307.52  
26                           1.06           37307.52  

[5 rows x 22 columns]

In [128]:
df.loc[(df['City'] == 'Hyderabad') & (df['Mapped_Sales_Type'] == 'Delivery'), :].head()

City Mapped_Sales_Type Mapped_Item_Code Transaction_Date  \
5   Hyderabad          Delivery             4056       2018-07-31   
10  Hyderabad          Delivery             4061       2018-07-31   
26  Hyderabad          Delivery             4068       2018-07-31   
42  Hyderabad          Delivery             4058       2018-07-31   
46  Hyderabad          Delivery             4064       2018-07-31   

    Quantity_Sold  Median_Price  Average_Tax_Percent  Effective_Price  \
5             617         322.0                 0.05       305.354585   
10            144         942.0                 0.05       942.631515   
26            148         622.0                 0.05       618.898649   
42            532         272.0                 0.05       252.660214   
46            261         682.0                 0.05       683.031933   

    Percent_Quantity_With_Discount  Avg_Discount_Percent_On_Discounted_Items  \
5                         0.003953                                  0.105020   
10                        0.151515                                  0.105006   
26                        0.027027                                  0.105009   
42                        0.014252                                  0.105008   
46                        0.006667                                  0.105008   

          ...          Day  Car_sales F_and_B_Inflation  \
5         ...          Tue    7559.28             143.7   
10        ...          Tue    7559.28             143.7   
26        ...          Tue    7559.28             143.7   
42        ...          Tue    7559.28             143.7   
46        ...          Tue    7559.28             143.7   

    F_and_B_Inflation_Adjusted_Effective_Price  \
5                                     1.818751   
10                                    6.253525   
26                                    4.000686   
42                                    1.452054   
46                                    4.446986   

    F_and_B_Inflation_Adjusted_Median_Price  First_Week_Of_The_Month_Dummy  \
5                                  1.732777                              0   
10                                 6.047321                              0   
26                                 3.820459                              0   
42                                 1.384830                              0   
46                                 4.237996                              0   

    Last_Week_Of_The_Month_Dummy  Day_Of_The_Week_Seasonality  \
5                              1                         0.82   
10                             1                         0.75   
26                             1                         0.64   
42                             1                         0.72   
46                             1                         0.76   

    Month_Of_The_Year_Seasonality  Two_Wheeler_sales  
5                            1.06           37307.52  
10                           1.06           37307.52  
26                           1.06           37307.52  
42                           1.06           37307.52  
46                           1.06           37307.52  

[5 rows x 22 columns]

In [129]:
df.loc[df['Day'].isin(['Mon', 'Tue']), :].head()

City Mapped_Sales_Type Mapped_Item_Code Transaction_Date  \
5   Hyderabad          Delivery             4056       2018-07-31   
10  Hyderabad          Delivery             4061       2018-07-31   
20  Hyderabad           Dine-in              966       2018-07-31   
24  Hyderabad           Dine-in              965       2018-07-31   
26  Hyderabad          Delivery             4068       2018-07-31   

    Quantity_Sold  Median_Price  Average_Tax_Percent  Effective_Price  \
5             617         322.0                 0.05       305.354585   
10            144         942.0                 0.05       942.631515   
20           1250         392.0                 0.05       378.840053   
24           1934         362.0                 0.05       347.258327   
26            148         622.0                 0.05       618.898649   

    Percent_Quantity_With_Discount  Avg_Discount_Percent_On_Discounted_Items  \
5                         0.003953                                  0.105020   
10                        0.151515                                  0.105006   
20                        0.003512                                  0.105008   
24                        0.001646                                  0.420000   
26                        0.027027                                  0.105009   

          ...          Day  Car_sales F_and_B_Inflation  \
5         ...          Tue    7559.28             143.7   
10        ...          Tue    7559.28             143.7   
20        ...          Tue    7559.28             143.7   
24        ...          Tue    7559.28             143.7   
26        ...          Tue    7559.28             143.7   

    F_and_B_Inflation_Adjusted_Effective_Price  \
5                                     1.818751   
10                                    6.253525   
20                                    2.330133   
24                                    2.110357   
26                                    4.000686   

    F_and_B_Inflation_Adjusted_Median_Price  First_Week_Of_The_Month_Dummy  \
5                                  1.732777                              0   
10                                 6.047321                              0   
20                                 2.219903                              0   
24                                 2.011134                              0   
26                                 3.820459                              0   

    Last_Week_Of_The_Month_Dummy  Day_Of_The_Week_Seasonality  \
5                              1                         0.82   
10                             1                         0.75   
20                             1                         0.92   
24                             1                         0.89   
26                             1                         0.64   

    Month_Of_The_Year_Seasonality  Two_Wheeler_sales  
5                            1.06           37307.52  
10                           1.06           37307.52  
20                           1.09           37307.52  
24                           1.09           37307.52  
26                           1.06           37307.52  

[5 rows x 22 columns]

In [130]:
df.loc[df['Percent_Quantity_With_Discount'] != 0.0, :].head()

City Mapped_Sales_Type Mapped_Item_Code Transaction_Date  \
5   Hyderabad          Delivery             4056       2018-07-31   
10  Hyderabad          Delivery             4061       2018-07-31   
20  Hyderabad           Dine-in              966       2018-07-31   
24  Hyderabad           Dine-in              965       2018-07-31   
26  Hyderabad          Delivery             4068       2018-07-31   

    Quantity_Sold  Median_Price  Average_Tax_Percent  Effective_Price  \
5             617         322.0                 0.05       305.354585   
10            144         942.0                 0.05       942.631515   
20           1250         392.0                 0.05       378.840053   
24           1934         362.0                 0.05       347.258327   
26            148         622.0                 0.05       618.898649   

    Percent_Quantity_With_Discount  Avg_Discount_Percent_On_Discounted_Items  \
5                         0.003953                                  0.105020   
10                        0.151515                                  0.105006   
20                        0.003512                                  0.105008   
24                        0.001646                                  0.420000   
26                        0.027027                                  0.105009   

          ...          Day  Car_sales F_and_B_Inflation  \
5         ...          Tue    7559.28             143.7   
10        ...          Tue    7559.28             143.7   
20        ...          Tue    7559.28             143.7   
24        ...          Tue    7559.28             143.7   
26        ...          Tue    7559.28             143.7   

    F_and_B_Inflation_Adjusted_Effective_Price  \
5                                     1.818751   
10                                    6.253525   
20                                    2.330133   
24                                    2.110357   
26                                    4.000686   

    F_and_B_Inflation_Adjusted_Median_Price  First_Week_Of_The_Month_Dummy  \
5                                  1.732777                              0   
10                                 6.047321                              0   
20                                 2.219903                              0   
24                                 2.011134                              0   
26                                 3.820459                              0   

    Last_Week_Of_The_Month_Dummy  Day_Of_The_Week_Seasonality  \
5                              1                         0.82   
10                             1                         0.75   
20                             1                         0.92   
24                             1                         0.89   
26                             1                         0.64   

    Month_Of_The_Year_Seasonality  Two_Wheeler_sales  
5                            1.06           37307.52  
10                           1.06           37307.52  
20                           1.09           37307.52  
24                           1.09           37307.52  
26                           1.06           37307.52  

[5 rows x 22 columns]

In [131]:
df.columns

Index(['City', 'Mapped_Sales_Type', 'Mapped_Item_Code', 'Transaction_Date',
       'Quantity_Sold', 'Median_Price', 'Average_Tax_Percent',
       'Effective_Price', 'Percent_Quantity_With_Discount',
       'Avg_Discount_Percent_On_Discounted_Items', 'Year', 'Month', 'Day',
       'Car_sales', 'F_and_B_Inflation',
       'F_and_B_Inflation_Adjusted_Effective_Price',
       'F_and_B_Inflation_Adjusted_Median_Price',
       'First_Week_Of_The_Month_Dummy', 'Last_Week_Of_The_Month_Dummy',
       'Day_Of_The_Week_Seasonality', 'Month_Of_The_Year_Seasonality',
       'Two_Wheeler_sales'],
      dtype='object')

In [132]:
df['Range'] = np.where(df['Effective_Price']>500, 1, 0.5)

In [133]:
df.head()

City Mapped_Sales_Type Mapped_Item_Code Transaction_Date  \
5   Hyderabad          Delivery             4056       2018-07-31   
10  Hyderabad          Delivery             4061       2018-07-31   
20  Hyderabad           Dine-in              966       2018-07-31   
24  Hyderabad           Dine-in              965       2018-07-31   
26  Hyderabad          Delivery             4068       2018-07-31   

    Quantity_Sold  Median_Price  Average_Tax_Percent  Effective_Price  \
5             617         322.0                 0.05       305.354585   
10            144         942.0                 0.05       942.631515   
20           1250         392.0                 0.05       378.840053   
24           1934         362.0                 0.05       347.258327   
26            148         622.0                 0.05       618.898649   

    Percent_Quantity_With_Discount  Avg_Discount_Percent_On_Discounted_Items  \
5                         0.003953                                  0.105020   
10                        0.151515                                  0.105006   
20                        0.003512                                  0.105008   
24                        0.001646                                  0.420000   
26                        0.027027                                  0.105009   

    ...    Car_sales  F_and_B_Inflation  \
5   ...      7559.28              143.7   
10  ...      7559.28              143.7   
20  ...      7559.28              143.7   
24  ...      7559.28              143.7   
26  ...      7559.28              143.7   

   F_and_B_Inflation_Adjusted_Effective_Price  \
5                                    1.818751   
10                                   6.253525   
20                                   2.330133   
24                                   2.110357   
26                                   4.000686   

    F_and_B_Inflation_Adjusted_Median_Price  First_Week_Of_The_Month_Dummy  \
5                                  1.732777                              0   
10                                 6.047321                              0   
20                                 2.219903                              0   
24                                 2.011134                              0   
26                                 3.820459                              0   

    Last_Week_Of_The_Month_Dummy  Day_Of_The_Week_Seasonality  \
5                              1                         0.82   
10                             1                         0.75   
20                             1                         0.92   
24                             1                         0.89   
26                             1                         0.64   

    Month_Of_The_Year_Seasonality  Two_Wheeler_sales  Range  
5                            1.06           37307.52    0.5  
10                           1.06           37307.52    1.0  
20                           1.09           37307.52    0.5  
24                           1.09           37307.52    0.5  
26                           1.06           37307.52    1.0  

[5 rows x 23 columns]

In [134]:
df.columns

Index(['City', 'Mapped_Sales_Type', 'Mapped_Item_Code', 'Transaction_Date',
       'Quantity_Sold', 'Median_Price', 'Average_Tax_Percent',
       'Effective_Price', 'Percent_Quantity_With_Discount',
       'Avg_Discount_Percent_On_Discounted_Items', 'Year', 'Month', 'Day',
       'Car_sales', 'F_and_B_Inflation',
       'F_and_B_Inflation_Adjusted_Effective_Price',
       'F_and_B_Inflation_Adjusted_Median_Price',
       'First_Week_Of_The_Month_Dummy', 'Last_Week_Of_The_Month_Dummy',
       'Day_Of_The_Week_Seasonality', 'Month_Of_The_Year_Seasonality',
       'Two_Wheeler_sales', 'Range'],
      dtype='object')

In [135]:
x = df[['Mapped_Item_Code', 'Median_Price', 'Average_Tax_Percent',
       'Effective_Price', 'Percent_Quantity_With_Discount',
       'Avg_Discount_Percent_On_Discounted_Items','Car_sales', 'F_and_B_Inflation',
       'F_and_B_Inflation_Adjusted_Effective_Price',
       'F_and_B_Inflation_Adjusted_Median_Price']]

In [136]:
y= df['Quantity_Sold']

In [137]:
from sklearn.linear_model import LinearRegression

In [138]:
lr = LinearRegression()

In [139]:
from sklearn.model_selection import train_test_split

In [140]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 123)

In [141]:
lr = LinearRegression()

In [142]:
lr.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [143]:
y_pred = lr.predict(x_test)

In [144]:
from sklearn import metrics

In [145]:
metrics.mean_absolute_error(y_test, y_pred)

317.11283831381604

In [146]:
#Ordinary least squares

In [147]:
import statsmodels.formula.api as sm

In [148]:
ols_output = sm.OLS(y, x.astype(float)).fit()

In [149]:
ols_output.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          Quantity_Sold   R-squared:                       0.739
Model:                            OLS   Adj. R-squared:                  0.739
Method:                 Least Squares   F-statistic:                     2519.
Date:                Tue, 30 Apr 2019   Prob (F-statistic):               0.00
Time:                        21:15:10   Log-Likelihood:                -67530.
No. Observations:                8902   AIC:                         1.351e+05
Df Residuals:                    8892   BIC:                         1.352e+05
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
==============================================================================================================
                                                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------
Mapped_Item_Code                              -0.1602      0.004    -41.915      0.000      -0.168      -0.153
Median_Price                                  -1.8539      8.112     -0.229      0.819     -17.754      14.047
Average_Tax_Percent                        -1925.7522    145.180    -13.265      0.000   -2210.339   -1641.166
Effective_Price                               -0.3288      6.761     -0.049      0.961     -13.582      12.924
Percent_Quantity_With_Discount               -46.1437     30.000     -1.538      0.124    -104.950      12.663
Avg_Discount_Percent_On_Discounted_Items     755.2340     24.265     31.125      0.000     707.670     802.798
Car_sales                                     -0.0124      0.007     -1.697      0.090      -0.027       0.002
F_and_B_Inflation                             13.4197      2.130      6.299      0.000       9.244      17.596
F_and_B_Inflation_Adjusted_Effective_Price   150.5045    952.429      0.158      0.874   -1716.475    2017.484
F_and_B_Inflation_Adjusted_Median_Price       -7.1866   1143.171     -0.006      0.995   -2248.065    2233.692
==============================================================================
Omnibus:                     2489.636   Durbin-Watson:                   1.894
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             8969.941
Skew:                           1.375   Prob(JB):                         0.00
Kurtosis:                       7.077   Cond. No.                     2.36e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.36e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""